In [ ]:
# 기본 경로 설정
# ===============================================================
import os
import subprocess

PROJECT_NAME = "rag"

ROOT_DIR = os.getcwd()

try:
    from google.colab import drive, userdata
    IS_COLAB_MODE = True
    print("코랩 모드")

except ModuleNotFoundError as e:
    IS_COLAB_MODE = False
    ROOT_DIR = os.path.abspath(os.path.join(ROOT_DIR, ".."))
    os.environ["KMP_DUPLICATE_LIB_OK"] = "True"
    os.environ["TOKENIZERS_PARALLELISM"] = "false"
    print(f"로컬 모드")


        DATA_DIR = os.path.join(ROOT_DIR, "data")
        RAW_DIR = os.path.join(DATA_DIR, "raw")
        os.makedirs(DATA_DIR, exist_ok=True)


# 환경변수 로드 설정
def get_secret(key_name: str):
    if IS_COLAB_MODE:
        return userdata.get(key_name)
    else:
        from dotenv import load_dotenv
        load_dotenv(dotenv_path=os.path.join(ROOT_DIR, ".env"))
        return os.getenv(key_name)


if IS_COLAB_MODE:

    drive.mount('/content/drive')

    DATA_DIR = os.path.join(ROOT_DIR, "drive", "MyDrive", "codeit-part3-team4", "data")
    RAW_DIR = os.path.join(DATA_DIR, "raw")
    os.makedirs(DATA_DIR, exist_ok=True)

    SAVE_DIR = os.path.join(DATA_DIR, "runs", PROJECT_NAME)

    os.makedirs(SAVE_DIR, exist_ok=True)


# .DS_Store 파일 삭제
_ = subprocess.run(
    ["find", DATA_DIR, "-name", ".DS_Store", "-type", "f", "-delete"],
    check=True
)

로컬 모드


In [ ]:
from glob import glob

DATA_CSV_PATH, DATA_XLSX_PATH, files_dir = glob(os.path.join(RAW_DIR, "*"))

FILE_PATH_LIST = glob(os.path.join(files_dir, "*"))

In [ ]:
import pandas as pd

xlsx_df = pd.read_excel(DATA_XLSX_PATH)
csv_df = pd.read_csv(DATA_CSV_PATH)

compare_df = csv_df == xlsx_df

In [ ]:
for column in compare_df.columns:
    print(column, compare_df[column].sum())

공고 번호 9
공고 차수 82
사업명 99
사업 금액 99
발주 기관 100
공개 일자 100
입찰 참여 시작일 74
입찰 참여 마감일 92
사업 요약 100
파일형식 100
파일명 100
텍스트 5


In [ ]:
extension_list = [file.split(".")[-1] for file in FILE_PATH_LIST]
set(extension_list)

{'docx', 'hwp', 'pdf'}

In [ ]:
PDF_PATH_LIST = [file for file in FILE_PATH_LIST if file.split(".")[-1] == "pdf"]
HWP_PATH_LIST = [file for file in FILE_PATH_LIST if file.split(".")[-1] == "hwp"]
DOCX_PATH_LIST = [file for file in FILE_PATH_LIST if file.split(".")[-1] == "docx"]

In [ ]:
import olefile

f = olefile.OleFileIO(HWP_PATH_LIST[0])
encoded_text = f.openstream("PrvText").read()
decoded_text = encoded_text.decode("UTF-16")

In [ ]:
decoded_text

'\r\n\r\n<『아세안+3 식량안보정보시스템 3단계 협력사업(캄보디아)』 PMC 용역 제안요청서>\r\n\r\n2025. 1.\r\n\r\n<><한국농어촌공사>\r\n<<><><><목   차><><><   > Ⅰ. 사업개요  01    1. 사업대상지 지도  01     2. 사업개요서 02  Ⅱ. 제안 요청내용  04    1. 용역개요  04     2. 기본방향  06    3. 세부 과업지침  09   Ⅲ. 입찰 안내사항  40    1. 입찰참가자격  40    2. 입찰 및 낙찰자 선정  41    3. 입찰참가 등록 및 입찰서 제출   42    4. 입찰시 유의사항   43    5. 청렴계약이행 준수   44    6. 제안서 평가 및 용역기관 선정   44    7. 기타사항   48    8. 최종낙찰자 유의사항   49 Ⅳ. 제안서 작성 안내사항  53    1. 제안 안내사항 53    2. 제안서 작성방법 56>\r\n< Ⅴ. 각종기준 등  63    1. 정량평가 기준  63    2. 정성평가 기준  75    3. 용역비 산정, 집행 및 정산기준  77    4. 전문가파견 가이드라인  98 Ⅵ. 서식  104     [별지 제1호 서식] 정성, 정량제안서(표지)  105     [별지 제2호 서식] 일반현황 및 연혁  107     [별지 제3호 서식] 회사 유사사업 수행실적  108     [별지 제4호 서식] 사업책임자(PM) 유사사업 수행실적  109     [별지 제5호 서식] 참여전문가의 자격사항(요약)  110     [별지 제6호 서식] 사업책임자(PM) 이력사항  111     [별지 제7호 서식] 참여전문가 이력사항  112     [별지 제8호 서식] 과업수행조직표  113     [별지 제9호 서식] 동의서  114     [별지 제10호 서식] 서약서  115     [별지 제11호 서식] 용역수행 실적증명서  116     [별지 제12호 서식] 계량평가 점수 자가 진단표  117     [별지 제13

In [ ]:
import copy
test = copy.copy(decoded_text)

test = test.replace("\n", "").replace("\r", "").replace(">", "")

In [ ]:
items = test.split("<")

test_df = pd.DataFrame(items).transpose()

In [ ]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,,『아세안+3 식량안보정보시스템 3단계 협력사업(캄보디아)』 PMC 용역 제안요청서2...,,한국농어촌공사,,,,,목 차,,,Ⅰ. 사업개요 01 1. 사업대상지 지도 01 2. 사업개요...,Ⅴ. 각종기준 등 63 1. 정량평가 기준 63 2. 정성평가 기준...
